In [1]:
import glob
import pandas as pd
import scipy.constants as scc
from scipy.optimize import curve_fit
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
import scipy
import scipy.stats as stats
from scipy.integrate import quad
from matplotlib.offsetbox import AnchoredText
import os

In [2]:
import matplotlib

font = {
        'size'   : 13,
        'family':'Serif',}

matplotlib.rc('font', **font)
matplotlib.rcParams['xtick.direction'] = 'in'
matplotlib.rcParams['ytick.direction'] = 'in'

Symbolic link: \
`ln -s *_full_table.fits *_irac_phot.fits`
\
`ln -s *_full_table.fits *_irac_phot_apcorr.fits`

In [3]:
import grizli

from grizli.pipeline import photoz
import numpy as np

#os.chdir('/home/oyster/OneDrive/thesis')


In [4]:
import main_body

In [5]:
import time
import multiprocessing as mp
import multiprocessing.pool

class NoDaemonProcess(mp.Process):
    # make 'daemon' attribute always return False
    def _get_daemon(self):
        return False
    def _set_daemon(self, value):
        pass
    daemon = property(_get_daemon, _set_daemon)

class NonDaemonPool(multiprocessing.pool.Pool):
    def Process(self, *args, **kwds):
        proc = super(NonDaemonPool, self).Process(*args, **kwds)

        class NonDaemonProcess(proc.__class__):
            """Monkey-patch process to ensure it is never daemonized"""
            @property
            def daemon(self):
                return False

            @daemon.setter
            def daemon(self, val):
                pass

        proc.__class__ = NonDaemonProcess
        return proc


path='/home/yjm/miniconda/Outputs'
os.chdir(path)
files_list = glob.glob('*')
if __name__ == '__main__':
    tic = time.time()
    pool = NonDaemonPool(3) 
    pool.map(main_body.main, files_list)
    pool.close()
    pool.join()
    toc = time.time()
    print('Done in {:.4f} seconds'.format(toc-tic))

Read default param file: j003212p1808_irac_masked.eazypy.zphot.param
Read CATALOG_FILE: j003212p1808_irac_masked_phot_apcorr.fits
   >>> NOBJ = 2445
f105w_tot_2 f105w_etot_2 (202): hst/wfc3/IR/f105w.dat
f125w_tot_2 f125w_etot_2 (203): hst/wfc3/IR/f125w.dat
f140w_tot_2 f140w_etot_2 (204): hst/wfc3/IR/f140w.dat
f160w_tot_2 f160w_etot_2 (205): hst/wfc3/IR/f160w.dat
f435w_tot_2 f435w_etot_2 (233): hst/ACS_update_sep07/wfc_f435w_t81.dat
f606w_tot_2 f606w_etot_2 (236): hst/ACS_update_sep07/wfc_f606w_t81.dat
f814w_tot_2 f814w_etot_2 (239): hst/ACS_update_sep07/wfc_f814w_t81.dat
irac_ch1_flux irac_ch1_err ( 18): IRAC/irac_tr1_2004-08-09.dat
irac_ch2_flux irac_ch2_err ( 19): IRAC/irac_tr2_2004-08-09.dat
Set sys_err = 0.05 (positive=True)
Template grid: templates/fsps_full/tweak_fsps_QSF_12_v3.param (this may take some time)


100%|███████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.37it/s]


Template   0: tweak_fsps_QSF_12_v3_001.dat (NZ=1).
Template   1: tweak_fsps_QSF_12_v3_002.dat (NZ=1).
Template   2: tweak_fsps_QSF_12_v3_003.dat (NZ=1).
Template   3: tweak_fsps_QSF_12_v3_004.dat (NZ=1).
Template   4: tweak_fsps_QSF_12_v3_005.dat (NZ=1).
Template   5: tweak_fsps_QSF_12_v3_006.dat (NZ=1).
Template   6: tweak_fsps_QSF_12_v3_007.dat (NZ=1).
Template   7: tweak_fsps_QSF_12_v3_008.dat (NZ=1).
Template   8: tweak_fsps_QSF_12_v3_009.dat (NZ=1).
Template   9: tweak_fsps_QSF_12_v3_010.dat (NZ=1).
Template  10: tweak_fsps_QSF_12_v3_011.dat (NZ=1).
Template  11: tweak_fsps_QSF_12_v3_012.dat (NZ=1).
Process templates: 3.049 s


256it [00:00, 1778.98it/s]


Load products: j003212p1808_irac_masked.eazypy.zout.fits
 ... Fit templates at zout[z_phot] 
fit_best: 0.4 s (n_proc=1,  NOBJ=2278)


/home/yjm/miniconda/main_body.py:267: RuntimeWarning: divide by zero encountered in log10
  m_exp = int(np.log10((m_50/2e33) // 1))


r0032 is done
Done in 83.4138 seconds


**From Kennicutt(1998a):**
\
$\operatorname{SFR}\left(M_{\odot} y r^{-1}\right)=4.5 \times 10^{-44} L_{\mathrm{IR}}\left(\operatorname{erg} s^{-1}\right)=1.71 \times 10^{-10} L_{\mathrm{IR}}\left(L_{\odot}\right)$
\
**From Whitaker+2014, ApJ, 795, 104:** \
$\operatorname{SFR}\left[M_{\odot} \mathrm{yr}^{-1}\right]=1.09 \times 10^{-10}\left(L_{\mathrm{IR}}+2.2 L_{\mathrm{UV}}\right)\left[L_{\odot}\right]$, where $L_{\mathrm{UV}}=1.5 v L_{v, 2800}$

In [6]:
SFR_IR = 1.09e-10 * L_50
Sv_nuv = nuv_flux * 1e-29 # from uJy to erg/s/cm^2
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
d_L = cosmo.luminosity_distance(z).to(u.cm)
L_v = 4*3.14*d_L.value**2*Sv_nuv
# L_UV = 1.5 * nu(2800) *Lv(2800) 
L_UV = 1.5 * 3e8 / 2.8e-7 * L_v / 3.82e33

SFR_UV = 2.2 * 1.09e-10 * L_UV
print(SFR_UV)
print(L_UV)

NameError: name 'L_50' is not defined

In [ ]:
fig, axes = plt.subplots(figsize=(20,5))
axes.axis('off')
ax = fig.add_subplot(131)
ax.scatter(M_50/2e33, M_50/2e33/M_stellar*100)
for i, txt in enumerate(ids):
    ax.annotate(txt, (M_50[i]/2e33, M_50[i]/M_stellar[i]/2e33*100))
ax.set_xlabel(r'$\mu (M_{dust}) \ /  \ M_{\odot}  $')
ax.set_ylabel(r'$\frac{M_{dust}}{M_{*}}\times $100%')

ax = fig.add_subplot(132)
ax.scatter(M_50/2e33, Av)
ax.set_xscale('log')
for i, txt in enumerate(ids):
    ax.annotate(txt, (M_50[i]/2e33, Av[i]))
ax.set_xlabel(r'$\mu (M_{dust}) \ /  \ M_{\odot}  $')
ax.set_ylabel(r'$Av$')

ax = fig.add_subplot(133)
x = np.arange(0,500)
y = x
ax.scatter(SFR_IR+SFR_UV, sfr)
ax.set_ylim(0,200)
ax.plot(x, y, label=r'$SFR_{eazy} = SFR_{IR}+SFR_{UV}$')
for i, txt in enumerate(ids):
    ax.annotate(txt, (SFR_IR[i]+SFR_UV[i], sfr[i]))
ax.set_xlabel(r'$\mu(SFR_{IR} + SFR_{UV}) \ / \ [M_{\odot} / yr]  $')
ax.set_ylabel(r'$SFR_{eazy}\ / \ [M_{\odot} / yr]$')
ax.legend()

In [ ]:
np.set_printoptions(precision=2)

difference16 = M_50 - M_16

difference84 = M_84 - M_50
df = pd.DataFrame({'ID': ids,
                    'M(50%) / M_sun': M_50/2e33,
                   'M(50%)-M(16%) / M_sun': difference16/2e33,
                   'M(84%)-M(50%) / M_sun': difference84/2e33,
                   'L_IR(50%) / L_sun':L_50 ,
                   'L_IR(50%)-L_IR(16%) / L_sun':L_50 - L_16,
                   'L_IR(84%)-L_IR(50%) / L_sun':L_84 - L_50
                  })
df.to_csv(path.split('/')[-1]+'_dustmass_L_IR.txt', sep='\t')

In [ ]:
print(L_16)
print(L_50)
print(L_50 - L_16)

In [ ]:
print(L_50)